# code contests exploration

## setup

In [ ]:
!pip install matplotlib
!pip install jupysql
!pip install pandas-profiling
!pip install ipywidgets

In [ ]:
%load_ext sql

In [ ]:
import sys
sys.path.append ("..")
from importlib import reload
from alpha_codium.code_contests.data import provider
import matplotlib.pyplot as plt
reload(provider)

import duckdb
from alpha_codium.code_contests.data.provider import CodeContestDataProvider

connection = duckdb.connect()

In [ ]:
""" pretty print problem and solution"""
import json
from IPython.display import display, Markdown
from black import format_str, FileMode
import re

def format_examples(description):
    # Check if 'Examples' section exists
    if 'Examples' not in description:
        return description
    
    # Split the description into main description and examples
    main_description, examples_section = description.split('Examples', 1)
    
    # Extract all the inputs and outputs
    inputs = re.findall(r'Input\n\n([\s\S]*?)(?:Output|$)', examples_section)
    outputs = re.findall(r'Output\n\n([\s\S]*?)(?:Input|$)', examples_section)
    
    # Create markdown table
    table = '### Examples\n\n| Input | Output |\n|-------|--------|\n'
    for inp, out in zip(inputs, outputs):
        table += f'| {inp.strip()} | {out.strip()} |\n'
    
    # Combine main description and table
    formatted_description = main_description + table
    return formatted_description

def render_problem_solution(problem_json, to_display=False):
    # Extract data from JSON
    name = problem_json.get('name', '')
    description = format_examples(problem_json.get('description', ''))
    solution = problem_json.get('solution', '')
    
    # Format the solution using black
    formatted_solution = format_str(solution, mode=FileMode())
    
    # Construct the markdown string
    markdown_str = f"""
## {name}

### description
{description}

### solution
```python
{formatted_solution}
```
"""
    if to_display:
        display(Markdown(markdown_str))

    else:
        return markdown_str
    # Display using Markdown

def render_problem_set(problems, to_display=True):
    sep = '<hr style="height:5px;border-width:0;color:black;background-color:gray">\n'
    markdown_str = f"""
# Problem set: length = {len(problems)}
{sep}
{sep.join([render_problem_solution(problem, False) for problem in problems])}
"""
    if to_display:
        display(Markdown(markdown_str))

    else:
        return markdown_str
    

In [ ]:
""" basic plot"""

import pandas as pd
import matplotlib.pyplot as plt

def plot_by_source(df, source_col, x_col, y_col, title = None):
    sources = df[source_col].unique()
    
    # Create subplots
    fig, axes = plt.subplots(nrows=len(sources), figsize=(12, 4 * len(sources)))

    # Check if there's only one source (axes won't be an array in that case)
    if len(sources) == 1:
        axes = [axes]
    
    for ax, source in zip(axes, sources):
        subset = df[df[source_col] == source]
        ax.bar(subset[x_col].astype(str), subset[y_col], color='skyblue', edgecolor='black')
        ax.set_title( f'{title or y_col} for {source_col}: {source}')
        ax.set_xlabel(x_col)
        ax.set_ylabel(y_col)
        ax.set_xticks(subset[x_col])
        ax.set_xticklabels(subset[x_col].astype(str), rotation=45)
    fig.suptitle(title, fontsize=16, y=1.03)
    plt.tight_layout()
    plt.show()

### Load the dataset and translate column codes

In [ ]:

cc = CodeContestDataProvider()
translated = cc.translate_references(cc.dataset)


In [ ]:
cc.connect(translated)
cc.query("SHOW ALL TABLES")

In [ ]:
print(json.dumps(translated['train'][0], indent=4))

## Splits and sources

In [ ]:
sizes = pd.Series({ds_split:len(ds_value) for ds_split, ds_value in translated.items()})
sizes

In [ ]:
split_distribution = []
for split in translated.keys():
    df = cc.query(f"select source, count(*) as num_samples from code_contests_{split} group by source order by source asc")
    df['split']=split
    split_distribution.append(df)

distribution = pd.concat(split_distribution, ignore_index=True)

plot_by_source(distribution, 'split', 'source', 'num_samples', "Source distribution by dataset split")


In [ ]:

flattened = translated['train'].flatten()
for col_name in flattened.column_names:
    new_name = col_name.replace(".", "__")
    if not col_name == new_name:
        flattened = flattened.rename_column(col_name,new_name )
                      
connection = cc.connection
connection.register(f"train_flattened", flattened.data.table)
schema = connection.query("show table train_flattened").df()
import pandas as pd
pd.set_option('display.max_rows', 50)
schema

## Solutions 

In [ ]:
""" distribution of tests per source """ 

def bucketized_group_by(table_name, field, measure):
    query = f"""
    SELECT
        {field}, 
        CASE
            WHEN counter =0  THEN '0'
            WHEN counter BETWEEN 1 AND 10 THEN '1-10'
            WHEN counter BETWEEN 11 AND 50 THEN '11-50'
            WHEN counter BETWEEN 51 AND 100 THEN '51-100'
            WHEN counter BETWEEN 101 AND 500 THEN '101-500'
            WHEN counter BETWEEN 501 AND 1000 THEN '501-1000'
        END AS bucket_range,
        COUNT(*) AS counter_bucket
    FROM
        (select {field}, {measure} as counter from {table_name}) as temp
    GROUP BY
        source,
        CASE
            WHEN counter = 0 THEN '0'
            WHEN counter BETWEEN 1 AND 10 THEN '1-10'
            WHEN counter BETWEEN 11 AND 50 THEN '11-50'
            WHEN counter BETWEEN 51 AND 100 THEN '51-100'
            WHEN counter BETWEEN 101 AND 500 THEN '101-500'
            WHEN counter BETWEEN 501 AND 1000 THEN '501-1000'
        END
    ORDER BY
        source desc,
            CASE
            WHEN bucket_range = '0' THEN 0
            WHEN bucket_range = '1-10' THEN 1
            WHEN bucket_range = '11-50' THEN 11
            WHEN bucket_range = '51-100' THEN 51
            WHEN bucket_range = '101-500' THEN 101
            WHEN bucket_range = '501-1000' THEN 501
        END ASC;
        
    """
    return query




In [ ]:
source_value = "CODEFORCES"  # Replace this with the desired value

query = f"""
SELECT 
    name, 
    description, 
    UNNEST(solutions__language) as language, 
    UNNEST(solutions__solution) as solution 
FROM 
    train_flattened 
WHERE 
    source='{source_value}'
"""

solutions = connection.query(query)
connection.register(f"train_solutions", solutions)
solutions

In [ ]:
language_distribution = connection.query(f"select '{source_value}' as source, language, count(*) as num_solutions from train_solutions group by language").df()

plot_by_source(language_distribution, 'source', 'language', 'num_solutions', "Solution languages by source in training set")


In [ ]:
solutions_sample = connection.query("select * from train_solutions USING SAMPLE 10%")
connection.register(f"train_solutions_sample", solutions_sample)


In [ ]:
language_value = "PYTHON3"  # Replace this with the desired value

query = f"""
SELECT 
    name, 
    count(*) as num_python_solutions_per_problem 
FROM 
    train_solutions_sample 
WHERE 
    language='{language_value}' 
GROUP BY 
    name
"""

solutions_per_problem = connection.query(query).df()
connection.register(f"train_solutions_per_problem", solutions_per_problem)
solutions_per_problem

## Tests 

In [ ]:
query = """
SELECT 
    name,
    source,
    len(public_tests__input) AS public_test_count,
    len(private_tests__input) AS private_test_count,
    len(generated_tests__input) AS generated_test_count
FROM 
    train_flattened;

"""

tests = connection.query(query)

connection.register(f"train_tests_per_problem", tests)

tests


In [ ]:
""" distribution of tests per source """ 
def test_count_per_source(field):
    test_count_distribution_query = bucketized_group_by('train_tests_per_problem', 'source', field )
    return connection.query(test_count_distribution_query).df()



In [ ]:
df = test_count_per_source('private_test_count')
plot_by_source(df, 'source', 'bucket_range', 'counter_bucket', title = "Distribution of private test counts in problems per source")


In [ ]:
df = test_count_per_source('generated_test_count')
plot_by_source(df, 'source', 'bucket_range', 'counter_bucket', title = "Distribution of generated test counts in problems per source")


In [ ]:
""" public tests - examples provided with the problem text itself """

query = """
SELECT 
    source, 
    public_test_count, 
    COUNT(*) AS num_of_problems
FROM 
    train_tests_per_problem
GROUP BY 
    source, 
    public_test_count
ORDER BY 
    source, 
    public_test_count;

"""
df =connection.query(query).df()

plot_by_source(df, 'source', 'public_test_count', 'num_of_problems', "Distribution of public (example) test counts in problems per source")

In [ ]:
connection.query("show all tables").df()

## Example visualization


In [ ]:
examples = connection.query ("select * from train_solutions_sample where language='PYTHON3' order by solution limit 10 ").df().to_dict(orient='records')


In [ ]:
render_problem_set(examples)